In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
df=pd.read_csv('2012-2013_cleaned2.csv', index_col=0)

In [3]:
df

,loan_amnt,term,int_rate,installment,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,...,fico_range_high,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,acc_open_past_24mths,mort_acc,percent_bc_gt_75,loan_status
0,12000.0,0.124432,7.62,373.94,32,Systems Engineer,3.0,0.142148,96500.0,0.122274,...,709.0,0.0,17.0,13248.0,55.7,30.0,4.00000,1.00000,100.000000,0.0
1,27050.0,0.124432,10.99,885.46,28,Team Leadern Customer Ops & Systems,10.0,0.158888,55000.0,0.177538,...,734.0,0.0,14.0,36638.0,61.2,27.0,3.00000,4.00000,25.000000,0.0
2,12000.0,0.124432,11.99,398.52,27,LTC,10.0,0.142148,130000.0,0.160987,...,719.0,1.0,9.0,10805.0,67.0,19.0,4.00000,3.00000,1.000000,0.0
3,28000.0,0.124432,7.62,872.52,32,Area Sales Manager,5.0,0.142148,325000.0,0.160987,...,749.0,1.0,15.0,29581.0,54.6,31.0,6.00000,5.00000,16.700000,0.0
4,27600.0,0.255994,19.97,730.78,15,Street Operations Supervisor,6.0,0.142148,73000.0,0.160987,...,669.0,1.0,10.0,27003.0,82.8,24.0,2.00000,4.00000,80.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180304,12000.0,0.124432,7.90,375.49,31,mpmi,3.0,0.142148,42000.0,0.122274,...,724.0,0.0,9.0,14378.0,52.3,14.0,3.92953,1.81102,53.558127,0.0
180305,20500.0,0.124432,16.77,728.54,18,NaN,7.0,0.172892,60000.0,0.160987,...,704.0,3.0,10.0,15417.0,58.4,20.0,3.92953,1.81102,53.558127,0.0
180306,15000.0,0.124432,15.27,521.97,21,NaN,3.0,0.172892,57600.0,0.160987,...,684.0,1.0,18.0,8897.0,33.1,30.0,3.92953,1.81102,53.558127,0.0
180307,35000.0,0.124432,15.96,1229.81,20,Tom and Holly Gores,3.0,0.142148,160000.0,0.160987,...,724.0,1.0,9.0,23665.0,62.4,17.0,3.92953,1.81102,53.558127,0.0


In [4]:
y=df['loan_status']
df.drop(columns=['emp_title','issue_d','title','zip_code','loan_status'], inplace=True)

In [7]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3)

In [8]:
# Save desc column for natural language processing latter
desc_train=X_train['desc']
desc_test=X_test['desc']

X_train.drop(columns=['desc'], inplace=True)
X_test.drop(columns=['desc'], inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
# Standardize the data
scaler=StandardScaler()
scaler.fit(X_train)
scaled_train=scaler.transform(X_train)
scaled_test=scaler.transform(X_test)

In [12]:
lmt = WordNetLemmatizer()
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation and digits
    2. Remove all stopwords
    3. Lemmatization
    4. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    mess=str(mess)
    mess=mess[mess.find('>')+1:mess.find('<')]
    nopunc = [char for char in mess if (char not in string.punctuation) and (not char.isdigit())]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    cleaned=[lmt.lemmatize(word.lower()) for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return ' '.join(cleaned)


In [19]:
desc_train=desc_train.apply(text_process)
desc_test=desc_test.apply(text_process)

In [26]:
vocab_size=2500
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(desc_train)
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'na': 2, 'credit': 3, 'card': 4, 'loan': 5, 'debt': 6, 'pay': 7, 'payment': 8, 'interest': 9, 'consolidate': 10, 'rate': 11, 'one': 12, 'year': 13, 'would': 14, 'high': 15, 'bill': 16, 'consolidation': 17, 'like': 18, 'get': 19, 'monthly': 20, 'paying': 21, 'want': 22, 'home': 23, 'lower': 24, 'help': 25, 'need': 26, 'month': 27, 'time': 28, 'money': 29, 'make': 30, 'thank': 31, 'used': 32, 'paid': 33, 'im': 34, 'free': 35, 'higher': 36, 'looking': 37, 'job': 38, 'balance': 39, 'use': 40, 'new': 41, 'plan': 42, 'payoff': 43, 'account': 44, 'car': 45, 'trying': 46, 'house': 47, 'lending': 48, 'club': 49, 'save': 50, 'expense': 51, 'much': 52, 'also': 53, 'consolidating': 54, 'good': 55, 'able': 56, 'never': 57, 'fund': 58, 'back': 59, 'improvement': 60, 'current': 61, 'medical': 62, 'two': 63, 'work': 64, 'amount': 65, 'using': 66, 'allow': 67, 'better': 68, 'purchase': 69, 'family': 70, 'stable': 71, 'start': 72, 'saving': 73, 'goal': 74, 'repair': 75, 'thanks': 76, 'finan

In [49]:
max_length=50
train_sequences = tokenizer.texts_to_sequences(desc_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
print(train_padded[1])
print(train_padded.shape)

[318   5  22   7 151  12   8 122  27  55   3 398  20   8  14  18  24  11
  34  21  61  11  31   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
(126216, 50)


In [50]:
test_sequences = tokenizer.texts_to_sequences(desc_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [55]:
embedding_dim=64
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [56]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [62]:
history = model.fit(train_padded, np.array(y_train), epochs=20,a
                    validation_data=(test_padded,np.array(y_test)))


Train on 126216 samples, validate on 54093 samples
Epoch 1/20
126216/126216 [==============================] - 215s 2ms/sample - loss: 0.4311 - accuracy: 0.8452 - val_loss: 0.4362 - val_accuracy: 0.8411
Epoch 2/20
126216/126216 [==============================] - 207s 2ms/sample - loss: 0.4265 - accuracy: 0.8454 - val_loss: 0.4371 - val_accuracy: 0.8411
Epoch 3/20
126216/126216 [==============================] - 225s 2ms/sample - loss: 0.4230 - accuracy: 0.8454 - val_loss: 0.4371 - val_accuracy: 0.8411
Epoch 4/20
126216/126216 [==============================] - 918s 7ms/sample - loss: 0.4191 - accuracy: 0.8462 - val_loss: 0.4385 - val_accuracy: 0.8408
Epoch 5/20
126216/126216 [==============================] - 220s 2ms/sample - loss: 0.4144 - accuracy: 0.8477 - val_loss: 0.4433 - val_accuracy: 0.8394
Epoch 6/20
126216/126216 [==============================] - 206s 2ms/sample - loss: 0.4091 - accuracy: 0.8494 - val_loss: 0.4474 - val_accuracy: 0.8378
Epoch 7/20
126216/126216 [===========